In [1]:
import os
import os.path as path
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la

from src.structured_random_features.src.models.weights import V1_weights

# Packages for fft and fitting data
from scipy import fftpack as fft
from sklearn.linear_model import Lasso

# Package for importing image representation
from PIL import Image, ImageOps


In [2]:
testmin = []
reformed = []



for i in range (10):
    # 1. Generate random V1 weights
    numCell = np.random.randint(50, 2000)
    dim = (30, 30)
    n = 30
    cellSize = np.random.randint(1, 10)
    sparseFreq = np.random.randint(1, 10)

    # Store generated V1 cells in W
    W = V1_weights(numCell, dim, cellSize, sparseFreq) 
    
    
    imgArr = imgArr.flatten().reshape((900, 1)) # Reshape image for dot product with W
    print("W shape:", W.shape, "\nimageArr Shape:", imgArr.shape)
    
    ## Get y
    y = W @ imgArr

    ## WΨ
    theta = fft.dctn(W.reshape(numCell, n, n), norm = 'ortho', axes = [1, 2])
    theta = theta.reshape(numCell, n*n)
    
    ## Initialize Lasso and Fit data
    mini = Lasso(alpha = 1*numCell)
    mini.fit(theta, y)
    
    ## Retrieve sparse vector s
    s = mini.coef_
    
    # Reform the image using sparse vector s with inverse descrete cosine
    reform = fft.idctn(s.reshape(n, n), norm='ortho', axes=[0,1])
    
    ## Return reform image array
    
    
    

NameError: name 'imgArr' is not defined